In [98]:
# Modules Requires

# Common
import numpy as np
import pandas as pd
import time
import os
import glob

# Special tools
from operator import itemgetter
from itertools import chain

# Sklearn 
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.externals import joblib

# Scipy
from scipy.stats import mode

# Gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [7]:
# Fetch data
os.chdir('/Users/daniellee/Desktop/Kaggle/data/stackoverflow_data/part1_approach1_output/')
testX, trainX = map(pd.read_csv, glob.glob('*.csv')[:2])
testy = pd.read_csv('testy.csv')
trainy = pd.read_csv('trainy.csv')

In [15]:
train_y_set = pd.concat([trainX['combined'], trainy['category']], axis=1)

In [60]:
# Create a tagged document list containing tokenized content and title

d2v_models = {}

topic_labelized = []
for topic in train_y_set.category.unique():
    # series 
    train_subset = train_y_set.combined[train_y_set.category == topic]
    labelized = [TaggedDocument(content.split(), [topic + '_' + str(_id)]) for _id, content in enumerate(train_subset)]
    
    topic_labelized += labelized
    
model = Doc2Vec(topic_labelized, size=100, window=5, min_count=1, workers=4)
#labelized = [TaggedDocument(content.split(), [_id]) for content, _id in zip(data.text, data.title)]

In [596]:
# Class prediction

def d2v_predict(x):
    
    def infer(x):
        new_vector = model.infer_vector(x.split())
        pred = model.docvecs.most_similar([new_vector],topn=10)

        return mode([doc[:doc.find('_')] for doc, score in pred])
    
    return mode([infer(x) for i in range(15)])[0][0][0][0]

In [604]:
pred = testX['combined'].apply(d2v_predict)

/Users/daniellee/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:250: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [605]:
le = LabelEncoder()
y_encode = le.fit(trainy['category'])

test_pred_y = le.transform(pred)
test_true_y = le.transform(testy['category'])

testF1 = f1_score(test_pred_y, test_true_y, average='macro')

In [608]:
testF1

0.88278440780975742